# Utilisation d'un nouveau FSPM avec PlantFusion

On propose d'intégrer un nouveau FSPM, écrit en lpy, avec PlantFusion. Il s'agit d'un modèle expansion simple, tiré de l'exercice 10 des TD impulse de Gaëtan.

Dans ce notebook, on va construire une interface ou wrapper ou façade, à ce modèle pour le connecter avec les modèles de lumière, de sol et même le planter.

## Premier test du FSPM seul

Notre FSPM se prénomme sobrement "plante rampante". La plante part d'une origine et fait une expansion à chaque pas de temps en fonction de la lumière captée. 

Une procédure pour faire varier la quantité de racine est implémentée pour l'exercice, mais biologiquement fausse.

In [ ]:
# Notre FSPM est codé en lpy
from openalea.lpy import *
import openalea.lpy as lpy

# pour la visualisation
from pgljupyter import LsystemWidget, SceneWidget

In [ ]:
name_lsystem = 'plante_rampante_aerien.lpy' 
lsystem = lpy.Lsystem(name_lsystem)

# simulation avec couplage lumiere
lsystem.opt_external_coupling = 0

LsystemWidget(name_lsystem, unit='cm',size_world=0.2, animate=True)

## Interfaçage

Notre wrapper va prendre la forme d'une classe. La structure générique d'une façade pour un FSPM doit avoir la forme:

```python
class FSPM_wrapper:
    def __init__(self, *arg) -> None:
        """
        Constructeur de l'instance, initialise le FSPM avec des préconfigurations
        
        """
        
    
    def light_inputs(self, *arg):
        """
        Renvoie une scène géométrique sous l'une de ces formes :
            - Scène PlantGL
            - MTG adelwheat
            - chemin d'un fichier VGX
            - dict de triangles rangés par organe
            - grille de voxels
        
        pour plus d'information : https://lightvegemanager.readthedocs.io/en/latest/inputs.html#scenes
        """
        
    
    def light_results(self, light_results, *arg) -> None:
        """
        Interprète les résultats de l'ensoleillement
        
        light_results : pandas.Dataframe
        """
        
    
    def soil_inputs(self, *arg):
        """
        Renvoie 4 tableaux :
            - teneur en N dans les racines pour chaque plante ([0, 1])
            - longeur des racines (en m) par plante et par voxel du sol
            - paramètres variétaux
            - capacité d'interception de la lumière par plante en fonction du couvert ([0, 1])
        """
        
    
    def soil_results(self, soil_results, *arg) -> None:
        """
        Interprète un ou plusieurs des 3 tableaux résutlats de soil3ds :
            - uptake N pour chaque plante (kg)
            - fraction d'eau disponible pour la transpiration par plante [0, 1]
            - quantite d'eau transpiree au temps t par plante (mm)
        """
        
    
    def run(self, *arg) -> None:
        """
        Encapsule les procédures non couplées (indépendantes) du FSPM
        
        """
        
    
    def end(self, *arg):
        """
        Optionnel
        
        Donne des instructions pour terminer la simulation au FSPM
        
        """

```

In [ ]:
# les deux outils pratiques de PlantFusion
from plantfusion.planter import Planter
from plantfusion.indexer import Indexer

# le module python associé à notre FSPM
from plante_rampante_functions import *

### Structure initiale de l'interface

On ajoute en plus une méthode ``derive`` liée à la particulité de lpy à dériver en début d'itération. En effet, les points de couplage doivent se trouver dans la section ``EndEach``.

In [ ]:
class PlanteRampante_Wrapper:
    def __init__(self, name="rampante", planter=Planter(), indexer=Indexer()) -> None:
        """Initialisation de l'interface

        Parameters
        ----------
        name : string
            nom personnalisé de notre instance de PlanteRampante
        planter : Planter, optional
            un planter pour récupère les dimensions xy du sol, by default Planter()
        indexer : Indexer, optional
            un indexer pour anticiper des simulations avec d'autres FSPM, by default Indexer()
        """ 
        
        self.name = "plantrampante"
        self.name_lsystem = 'plante_rampante_aerien.lpy' 
        self.lsystem = lpy.Lsystem(self.name_lsystem)

        self.indexer = indexer
        if indexer.global_order != [] :
            self.global_index = indexer.global_order.index(name)
        else:
            self.global_index = 0

        self.nb_plant = 1

        self.soilsurf = (planter.domain[1][0] - planter.domain[0][0]) ** 2

        # simulation avec couplage lumiere
        self.lsystem.opt_external_coupling = 1

        self.lstring = self.lsystem.axiom
        
        self.data_results = {"dMS" : [], "dMSRoot" : [], "epsi" : [], "roots_length" : [], "t" : []}


In [ ]:
class PlanteRampante_Wrapper(PlanteRampante_Wrapper):
    def derive(self, t):
        """Dérivation d'une itération

        Parameters
        ----------
        t : int
            pas de temps
        """        
        
        self.lstring = self.lsystem.derive(self.lstring, t, 1)
        self.data_results["t"].append(t)

### Gestion de la lumière

Scène géométrique pour calculer un ensoleillement. Dans lpy, la scène géométrique est contenue dans la section ``interpretation``.

In [ ]:
class PlanteRampante_Wrapper(PlanteRampante_Wrapper):
    def light_inputs(self):
        """
        Renvoie une scène géométrique sous l'une de ces formes :
            - Scène PlantGL
            - MTG adelwheat
            - chemin d'un fichier VGX
            - dict de triangles rangés par organe
            - grille de voxels
        
        pour plus d'information : https://lightvegemanager.readthedocs.io/en/latest/inputs.html#scenes
        """
        return self.lsystem.sceneInterpretation(self.lstring)

In [ ]:
# test de visualisation du point de départ
plane = ((-1., -1.), (0.1, 0.1))
planter = Planter(xy_plane=plane)
rampante = PlanteRampante_Wrapper(planter=planter)

# visualisation
SceneWidget(rampante.light_inputs(), 
                position=(0.0, 0.0, 0.0), 
                size_display=(600, 400), 
                plane=True, 
                size_world=2., 
                axes_helper=True)

Le tableau résultat prend la forme 

pour CARIBU:

| "Day" | "Hour" | "Organ" | "VegetationType" | "Area" | "par Eabs" | "par Ei" |
| --- | --- | --- | --- | --- | --- | --- |

pour RATP:

| "Day" | "Hour" | "Organ" | "VegetationType" | "Area" | "PARa" | "Intercepted" | "Transmitted" |
| --- | --- | --- | --- | --- | --- | --- | --- |




On ajoute les étapes calcul de l'assimiliation du carbone et calcul de l'interception des plantes car ces données seront utilisés par le traitement du sol.

In [ ]:
class PlanteRampante_Wrapper(PlanteRampante_Wrapper):
    def light_results(self, light_results_per_organ, energy) -> None:
        """
        Interprète les résultats de l'ensoleillement
        
        light_results_per_organ : pandas.Dataframe
        energy :
        
        aggregated_out = {
            "default_band": {
                     "Eabs" : { indice_organe : "par Eabs"} 
                     "Ei" :    { indice_organe : "par Ei"}
                     "area" : {indice_rogane : area_i}
            }
        }
        """
        organs_par = { "default_band" : { "Eabs" : {} , "Ei" : {}, "area" : {} } }
        
        df_filtered = light_results_per_organ[light_results_per_organ.VegetationType == self.global_index]
        for i in range(len(df_filtered)):
            organs_par["default_band"]["Eabs"][df_filtered.loc[i, "Organ"]] = df_filtered.loc[i, "par Eabs"]
            organs_par["default_band"]["Ei"][df_filtered.loc[i, "Organ"]] = df_filtered.loc[i, "par Ei"]
            organs_par["default_band"]["area"][df_filtered.loc[i, "Organ"]] = df_filtered.loc[i, "Area"]
        
        self.lstring, self.cumlight, ls_par = update_light_lstring(self.lstring, organs_par)

        #Carbon assimilation and allocation
        self.dMS = CarbonAssimilation(self.cumlight, energy, self.lsystem.RUE, self.lsystem.FTSW, Soilsurf=self.soilsurf)

        # plant light interception
        epsi =  self.cumlight / self.soilsurf
        self.ls_epsi = [epsi] * self.nb_plant # partage identique entre plantes

In [ ]:
import pandas

# test avec un tableau fictif
lighting = {"Day" : [1], 
            "Hour" : [12],  
            "Organ": [2], # indice de la graine
            "VegetationType": [0], 
            "Area" : [1.], 
            "par Eabs" : [0.5], 
            "par Ei" : [0.5]}
lighting_df = pandas.DataFrame(lighting)

# création d'une instance
plane = ((-1., -1.), (0.1, 0.1))
planter = Planter(xy_plane=plane)
rampante = PlanteRampante_Wrapper(planter=planter)
rampante.derive(0)

energy = 1.
rampante.light_results(lighting_df, energy)

# aucune feuille pour le moment
print(rampante.cumlight)

In [ ]:
import pandas

# test avec un tableau fictif
lighting = {"Day" : [1], 
            "Hour" : [12],  
            "Organ": [2], # indice de la graine
            "VegetationType": [0], 
            "Area" : [1.], 
            "par Eabs" : [0.5], 
            "par Ei" : [0.5]}
light_results_per_organ = pandas.DataFrame(lighting)
light_results_per_organ

### Gestion du sol

Ici, la teneur en azote des racines est statique et est initialisée dans le lsystem. Les paramètres variétaux liés au sol sont des valeurs par défaut donné par soil3ds et initialisées dans le lsystem.

In [ ]:
class PlanteRampante_Wrapper(PlanteRampante_Wrapper):
    def soil_inputs(self, soil_wrapper):
        """
        Renvoie 4 tableaux :
            - teneur en N dans les racines pour chaque plante ([0, 1])
            - longeur des racines (en m) par plante et par voxel du sol
            - paramètres variétaux
            - capacité d'interception de la lumière par plante en fonction du couvert ([0, 1])
        """
        self.roots_length = self.lsystem.roots_length
        
        roots_length_per_plant_per_soil_layer = roots_length_repartition(self.roots_length, 
                                                                        self.lsystem.carto, 
                                                                        soil_wrapper.soil.dxyz[0][0], 
                                                                        soil_wrapper.soil.origin, 
                                                                        soil_wrapper.soil_dimensions)

        N_content_roots_per_plant = self.lsystem.ls_N
        plants_soil_parameters = self.lsystem.ParamPN
        plants_light_interception = self.ls_epsi

        return (
            N_content_roots_per_plant,
            roots_length_per_plant_per_soil_layer,
            plants_soil_parameters,
            plants_light_interception,
        )

In [ ]:
# impression des listes
plane = ((-1., -1.), (0.1, 0.1))
planter = Planter(xy_plane=plane)

# création d'un sol test
from plantfusion.soil_wrapper import Soil_wrapper
soil = Soil_wrapper(in_folder="soil3ds_inputs", IDusm=1, planter=planter)

# création d'une instance
rampante = PlanteRampante_Wrapper(planter=planter)

# initialise l'interception de la plante pour éviter le calcul de lumière
rampante.ls_epsi = [0.1]

(N_content_roots_per_plant,
roots_length_per_plant_per_soil_layer,
plants_soil_parameters,
plants_light_interception) = rampante.soil_inputs(soil)

print(N_content_roots_per_plant)
print(roots_length_per_plant_per_soil_layer)
print(plants_soil_parameters)
print(plants_light_interception)

In [ ]:
class PlanteRampante_Wrapper(PlanteRampante_Wrapper):
    def soil_results(self, uptakeN_per_plant) -> None:
        """
        Interprète le tableau uptake N pour chaque plante (kg)
        """
        self.uptakeN = numpy.sum(uptakeN_per_plant[0])
        self.roots_length = growth_roots(self.roots_length, self.uptakeN, self.dMS)

In [ ]:
# vérification

# un petit planter
plane = ((-1., -1.), (0.1, 0.1))
planter = Planter(xy_plane=plane)

# création d'une instance
rampante = PlanteRampante_Wrapper(planter=planter)

# initialise cette variable pour éviter d'appeler derive
rampante.roots_length = 1.
rampante.dMS = 0.

uptakeN_per_plant = numpy.ones([2, 2, 2]) * 1.2
rampante.soil_results(uptakeN_per_plant)

print(rampante.roots_length)

### Complète l'interface
On ajoute des méthodes ``run`` et ``end`` à titre d'exmple. Ici, ``run`` transmets les grandeurs calculées au lsystem et ``end`` affiche dans un tableau toutes les grandeurs de la simulation

In [ ]:
class PlanteRampante_Wrapper(PlanteRampante_Wrapper):
    def run(self):
        self.lsystem.MS += self.dMS
        self.dMSRoot = self.dMS * self.lsystem.AllocRoot

        self.lsystem.root_length = self.roots_length
        self.lsystem.ls_epsi = self.ls_epsi

        self.data_results["dMS"].append(self.dMS)
        self.data_results["dMSRoot"].append(self.dMSRoot)
        self.data_results["epsi"].append(numpy.sum(self.ls_epsi))
        self.data_results["roots_length"].append(self.roots_length)

    def end(self):
        print(pandas.DataFrame(self.data_results))

## Premier essai avec PlantFusion

In [ ]:
# import des modules nécesssaires
from plantfusion.indexer import Indexer
from plantfusion.light_wrapper import Light_wrapper
from plantfusion.soil_wrapper import Soil_wrapper
from plantfusion.planter import Planter

#### initialisation des modules

1) Indexer

In [ ]:
name = "planterampante"
indexer = Indexer(global_order=[name], 
                  other_names=[name])

2) Planter

In [ ]:
plane = ((-1., -1.), (0.1, 0.1))
planter = Planter(xy_plane=plane)

3) façade de notre FSPM

In [ ]:
rampante = PlanteRampante_Wrapper(name, planter, indexer)

4) Lumière

In [ ]:
light = Light_wrapper(lightmodel="caribu", planter=planter, infinite=True)

5) Sol

In [ ]:
soil = Soil_wrapper(in_folder="soil3ds_inputs", IDusm=1, planter=planter)

#### Run de la simulation

variables à initialiser

In [ ]:
IncomingLight = 0.001 # MJ.m-2-degredays-1 (jour cumulant 10 degres days)
iterations = 100
scenes_gl = []

In [ ]:
# lancement de la simulation
doy_start = 60
for i in range(iterations):
    print("t: ",i)

    rampante.derive(i)

    # LIGHT #
    scene = rampante.light_inputs()
    light.run(scenes=[scene], day=i)
    rampante.light_results(light.results_organs(), energy=IncomingLight)
    
    scenes_gl.append(light.plantgl(lighting=True))

    # SOIL #
    soil_inputs_planterampante = rampante.soil_inputs(soil)
    
    (N_content_roots_per_plant,
    roots_length_per_plant_per_soil_layer,
    plants_soil_parameters,
    plants_light_interception) = indexer.soil_inputs({name : soil_inputs_planterampante})
    
    soil.run(
            day=doy_start+i,
            N_content_roots_per_plant=N_content_roots_per_plant,
            roots_length_per_plant_per_soil_layer=roots_length_per_plant_per_soil_layer,
            soil_plants_parameters=plants_soil_parameters,
            plants_light_interception=plants_light_interception,
        )
    soiltemporaire, stateEV, ls_ftsw, ls_transp, ls_Act_Nuptake_plt, temps_sol = soil.results
    rampante.soil_results(ls_Act_Nuptake_plt)
    
    rampante.run()    


bonus pour illustrer ce que peut faire la méthode end(), ici elle affiche des grandeurs

In [ ]:
rampante.end()

Visualisation de la scène en exportant les scènes plantGL

In [ ]:
# visualisation
t = 0
SceneWidget(scenes_gl[60], 
                position=(0.0, 0.0, 0.0), 
                size_display=(600, 400), 
                plane=True, 
                size_world=5., 
                axes_helper=True)

## Planter : Interprétation de la position des plantes
On va transférer la position de la plante calculée par le planter vers notre FSPM

In [ ]:
from soil3ds import soil_moduleN as solN

class PlanteRampante_Wrapper(PlanteRampante_Wrapper):
    def __init__(self, name="rampante", planter=Planter(), indexer=Indexer()) -> None:
        """Initialisation de l'interface

        Parameters
        ----------
        name : string
            nom personnalisé de notre instance de PlanteRampante
        planter : Planter, optional
            un planter pour récupère les dimensions xy du sol, by default Planter()
        indexer : Indexer, optional
            un indexer pour anticiper des simulations avec d'autres FSPM, by default Indexer()
        """ 
        
        self.name = "plantrampante"
        self.name_lsystem = 'plante_rampante_aerien.lpy' 
        self.lsystem = lpy.Lsystem(self.name_lsystem)

        self.indexer = indexer
        if indexer.global_order != [] :
            self.global_index = indexer.global_order.index(name)
        else:
            self.global_index = 0

        self.nb_plant = 1

        self.soilsurf = (planter.domain[1][0] - planter.domain[0][0]) ** 2
        
        # transfert de la position de la plante
        if planter.generation_type != "default":
            self.lsystem.carto = planter.generate_random_other()
            self.nb_plant = len(self.lsystem.carto)
            a = AxialTree()
            for i in range(self.nb_plant):
                a.append(self.lsystem.plante(i))
                a.append(self.lsystem.A(0, 1., 0))
            self.lsystem.axiom = a
        
            self.lsystem.nb_plt = self.nb_plant
            self.lsystem.ParamPN = [solN.default_paramp()] * self.nb_plant
            self.lsystem.ls_N = [1.] * self.nb_plant

        # simulation avec couplage lumiere
        self.lsystem.opt_external_coupling = 1

        self.lstring = self.lsystem.axiom
        
        self.data_results = {"dMS" : [], "dMSRoot" : [], "epsi" : [], "roots_length" : [], "t" : []}

In [ ]:
from plantfusion.indexer import Indexer
from plantfusion.light_wrapper import Light_wrapper
from plantfusion.soil_wrapper import Soil_wrapper
from plantfusion.planter import Planter

name = "planterampante"
indexer = Indexer(global_order=[name], other_names=[name])

plantdensity = {name : 10}
xy_square_length = 0.5
planter = Planter(generation_type="random", 
                  indexer=indexer, 
                  plant_density=plantdensity, 
                  xy_square_length=xy_square_length)
planter.transformations["scenes unit"][0] = "cm"

rampante = PlanteRampante_Wrapper(name, planter, indexer)

light = Light_wrapper(lightmodel="caribu", planter=planter, infinite=True)
soil = Soil_wrapper(in_folder="soil3ds_inputs", IDusm=1, planter=planter)

IncomingLight = 0.001 # MJ.m-2-degredays-1 (jour cumulant 10 degres days)
doy_start = 60
for i in range(iterations):
    print("t: ",i)

    rampante.derive(i)

    # LIGHT #
    scene = rampante.light_inputs()
    light.run(scenes=[scene], day=i)
    rampante.light_results(light.results_organs(), energy=IncomingLight)

    # SOIL #
    (
        N_content_roots_per_plant,
        roots_length_per_plant_per_soil_layer,
        soil_plants_parameters,
        plants_light_interception,
    ) = rampante.soil_inputs(soil)
    soil.run(
        day=doy_start+i,
        N_content_roots_per_plant=[N_content_roots_per_plant],
        roots_length_per_plant_per_soil_layer=[roots_length_per_plant_per_soil_layer],
        soil_plants_parameters=[soil_plants_parameters],
        plants_light_interception=[plants_light_interception],
    )
    rampante.soil_results(soil.results[4])

    rampante.run()

rampante.end()

In [ ]:
# visualisation
t = 1
SceneWidget(rampante.light_inputs(), 
                position=(0.0, 0.0, 0.0), 
                size_display=(600, 400), 
                plane=True, 
                size_world=5., 
                axes_helper=True)

In [ ]:
rampante.lsystem.carto

In [ ]:
rampante.lstring

## Plante rampante + CN-wheat

In [ ]:
def simulation_planterampante_wheat(iterations, in_folder, out_folder, write_geo):
    from plantfusion.indexer import Indexer
    from plantfusion.light_wrapper import Light_wrapper
    from plantfusion.soil_wrapper import Soil_wrapper
    from plantfusion.planter import Planter
    from plantfusion.wheat_wrapper import Wheat_wrapper

    # initialisation des noms et de l'indexer
    name1 = "planterampante"
    name2 = "wheat"
    indexer = Indexer(global_order=[name1, name2], wheat_names=[name2], other_names=[name1])
    
    # plantation des plantes
    plantdensity = {name1 : 10, name2 : 150}
    xy_square_length = 0.5
    planter = Planter(generation_type="random", indexer=indexer, plant_density=plantdensity, xy_square_length=xy_square_length)
    planter.transformations["scenes unit"][0] = "cm"
    # instance de plante rampante
    rampante = PlanteRampante_Wrapper(name1, planter, indexer)

    # paramètres d'entrée pour WheatFSPM
    RERmax_vegetative_stages_example = {
        "elongwheat": {
            "RERmax": {5: 3.35e-06, 6: 2.1e-06, 7: 2.0e-06, 8: 1.83e-06, 9: 1.8e-06, 10: 1.65e-06, 11: 1.56e-06}
        }
    }
    tillers_replications = {"T1": 0.5, "T2": 0.5, "T3": 0.5, "T4": 0.5}
    senescwheat_timestep = 1
    light_timestep = 4

    # instance de WheatFspm
    wheat = Wheat_wrapper(
        in_folder=in_folder,
        name=name2,
        out_folder=out_folder,
        planter=planter,
        indexer=indexer,
        external_soil_model=True,
        nitrates_uptake_forced=False,
        update_parameters_all_models=RERmax_vegetative_stages_example,
        tillers_replications=tillers_replications,
        SENESCWHEAT_TIMESTEP=senescwheat_timestep,
        LIGHT_TIMESTEP=light_timestep,
        SOIL_PARAMETERS_FILENAME="inputs_soil_legume/Parametres_plante_exemple.xls"
    )

    # instance pour la lumière
    light = Light_wrapper(lightmodel="caribu", indexer=indexer, planter=planter, out_folder=out_folder, infinite=True, writegeo=write_geo)
    
    # instance pour le sol
    soil = Soil_wrapper(in_folder="soil3ds_inputs", IDusm=1, planter=planter)

    # énergie constante pour plante rampante
    IncomingLight = 0.001 # MJ.m-2-degredays-1 (jour cumulant 10 degres days)
    # itérations de plante rampante
    t_pr = 0
    doy_start = 60
    
    # RUN SIMULATION #
    for t_wheat in range(wheat.start_time, iterations, wheat.SENESCWHEAT_TIMESTEP):
        ## conditions pour calculer l'environnement
        # on change de jour
        activate_planterampante = wheat.doy(t_wheat) != wheat.next_day_next_hour(t_wheat)
        # la soleil est levé
        daylight = (t_wheat % light_timestep == 0) and (wheat.PARi_next_hours(t_wheat) > 0)

        if daylight or activate_planterampante:
            # derive le lsystem avant la lumière et le sol
            if activate_planterampante:
                rampante.derive(t_pr)

            # LIGHT #
            scene1 = rampante.light_inputs()
            scene2, stems = wheat.light_inputs(planter)
            scenes = indexer.light_scenes_mgmt({name1 : scene1, name2 : scene2})
            light.run(
                scenes=scenes,
                day=wheat.doy(t_wheat),
                hour=wheat.hour(t_wheat),
                parunit="RG",
                stems=stems
            )

            # transmets la lumière à wheat
            if daylight:
                wheat.light_results(energy=wheat.energy(t_wheat), lighting=light)

            if activate_planterampante:
                # transmets la lumière à plante rampante
                rampante.light_results(light.results_organs(), energy=IncomingLight)

                # SOIL #
                soil_wheat_inputs = wheat.soil_inputs(soil, planter, light)
                soil_planterampante_inputs = rampante.soil_inputs(soil)
                (
                    N_content_roots_per_plant,
                    roots_length_per_plant_per_soil_layer,
                    plants_soil_parameters,
                    plants_light_interception,
                ) = indexer.soil_inputs({name1 : soil_planterampante_inputs, name2 : soil_wheat_inputs})
                soil.run(
                    day=doy_start+t_pr,
                    N_content_roots_per_plant=N_content_roots_per_plant,
                    roots_length_per_plant_per_soil_layer=roots_length_per_plant_per_soil_layer,
                    soil_plants_parameters=plants_soil_parameters,
                    plants_light_interception=plants_light_interception,
                )
                rampante.soil_results(soil.results[4])
                wheat.soil_results(soil.results[4], planter=planter)

                rampante.run()
                t_pr += 1
        
        wheat.run(t_wheat)
    
    rampante.end()

In [ ]:
iterations = 1000
in_folder_wheat = "inputs_fspmwheat"
out_folder = "outputs"
write_geo = True
simulation_planterampante_wheat(iterations, in_folder_wheat, out_folder, write_geo)